In [32]:
#!/usr/bin/env python
# encoding: utf-8

import chardet
import csv
import json
import os
import re
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import time
import zipfile
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join
from os.path import splitext
from pandas import DataFrame as df
from cimaq_utils import flatten
from cimaq_utils import loadimages
from NEW_extract_events2 import NEW_extract_events2
from chardet.universaldetector import UniversalDetector
# p1 = pd.read_csv("~/extracted_eprime_AS_FWF/participants.tsv", sep='\t')

def NEW_get_allsheetsdf(participants_sheet_path="~/extracted_eprime_AS_FWF/participants.tsv"):
    
    '''Generate DataFrame indexing participants and their respective files'''
    maindir = xpu(dname(participants_sheet_path))
    p1 = pd.read_csv("~/extracted_eprime_AS_FWF/participants.tsv",
                     sep='\t').set_index("sub-ID")[["sheetpaths"]]

    allsheets = sorted(flatten([[join(maindir, row[0], file)
                                 for file in ls(join(maindir, row[0]))
                                 if file.endswith(".txt") and \
                                  bool(file.find("MACOSX") != -1)==False]
                                for row in p1.iterrows()]))
    
    allsheets = sorted([(bname(dname(sheet)), sheet) for sheet in allsheets])
    allsheetsdf = df(allsheets, columns=['sub-ID', 'sheetpath']).set_index('sub-ID')
    allsheetsdf['prefix'] = [bname(row[1]['sheetpath']).split(\
                                row[0][4:4+len(row[0].split("sub-")[1])])[0]
                             for row in allsheetsdf.iterrows()]

    return allsheetsdf

def main():
    allsheetsdf = NEW_get_allsheetsdf()

if __name__ == '__main__':
    main()
    

In [34]:
allsheetsdf = NEW_get_allsheets()
display(allsheetsdf)

,sheetpath,prefix
sub-ID,,
sub-3025432,/home/fnadeau/extracted_eprime_AS_FWF/sub-3025...,Onset-Event-Encoding_CIMAQ_
sub-3025432,/home/fnadeau/extracted_eprime_AS_FWF/sub-3025...,Output-Responses-Encoding_CIMAQ_
sub-3025432,/home/fnadeau/extracted_eprime_AS_FWF/sub-3025...,Output_Retrieval_CIMAQ_
sub-3123186,/home/fnadeau/extracted_eprime_AS_FWF/sub-3123...,Onset-Event-Encoding_CIMAQ_
sub-3123186,/home/fnadeau/extracted_eprime_AS_FWF/sub-3123...,Output-Responses-Encoding_CIMAQ_
...,...,...
sub-9494076,/home/fnadeau/extracted_eprime_AS_FWF/sub-9494...,Output-Responses-Encoding_CIMAQ_
sub-9494076,/home/fnadeau/extracted_eprime_AS_FWF/sub-9494...,Output_Retrieval_CIMAQ_
sub-9929164,/home/fnadeau/extracted_eprime_AS_FWF/sub-9929...,Onset-Event-Encoding_CIMAQ_


In [39]:
#!/usr/bin/env python
# -*- coding: ISO-8859-1 -*-

import datetime
import chardet
import csv
import json
import os
import re
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import time
import zipfile
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join
from os.path import splitext
from pandas import DataFrame as df
from cimaq_utils import flatten
from cimaq_utils import loadimages
from NEW_extract_events2 import NEW_extract_events2
from NEW_get_allsheetsdf import NEW_get_allsheetsdf
from NEW_sniff_encodtype import NEW_sniff_encodtype
from chardet.universaldetector import UniversalDetector

def NEW_sniff_encodtype(maindir="~/extracted_eprime_AS_FWF"):
    ''' Detects encoding type used in given data table.
        Performs better with incremental runs.
        Source:
            https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
    '''
    if os.path.isdir(xpu(maindir)):
        allsheetsdf = NEW_get_allsheetsdf(join(maindir, 'participants.tsv'))
    else:
        allsheetsdf = allsheetsdf
    results = []
    for row in allsheetsdf.iterrows():
        detector = UniversalDetector()
        detector.reset()
        for line in open(row[1]['sheetpath'], 'rb'):
            detector.feed(line)
            if detector.done: break
        detector.close()
        results.append(pd.Series(detector.result, name=row[0]).squeeze())
    rezz = df(results).drop('language', axis=1)

    return rezz

def main():
    sniffed = NEW_sniff_encodtype()

if __name__ == '__main__':
    main()

In [41]:
rezz = NEW_sniff_encodtype()
display(rezz)

,encoding,confidence
sub-3025432,ascii,1.00
sub-3025432,ascii,1.00
sub-3025432,ISO-8859-1,0.73
sub-3123186,ascii,1.00
sub-3123186,ascii,1.00
...,...,...
sub-9494076,ascii,1.00
sub-9494076,ISO-8859-1,0.73
sub-9929164,ascii,1.00
sub-9929164,ascii,1.00


In [ ]:
#!/usr/bin/env python
# -*- coding: ISO-8859-1 -*-

import datetime
import chardet
import csv
import json
import os
import re
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import time
import zipfile
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join
from os.path import splitext
from pandas import DataFrame as df
from cimaq_utils import flatten
from cimaq_utils import loadimages
from NEW_extract_events2 import NEW_extract_events2
from NEW_get_allsheetsdf import NEW_get_allsheetsdf
from chardet.universaldetector import UniversalDetector

def NEW_sniff_encodtype(maindir="~/extracted_eprime_AS_FWF"):
    ''' Detects encoding type used in given data table.
        Performs better with incremental runs.
        Source:
            https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
    '''
    if maindir:
        allsheetsdf = NEW_get_allsheetsdf()
    else:
        allsheetsdf = NEW_get_allsheetsdf(join(maindir, 'participants.tsv'))
    results = []
    for row in allsheetsdf.iterrows():
        detector = UniversalDetector()
        detector.reset()
        for line in open(row[1]['sheetpath'], 'rb'):
            detector.feed(line)
            if detector.done: break
        detector.close()
        results.append(pd.Series(detector.result, name=row[0]).squeeze())
    rezz = df(results).drop('language', axis=1)

    return rezz

def main():
    sniffed = NEW_sniff_encodtype()

if __name__ == '__main__':
    main()

In [31]:
#!/usr/bin/env python
# -*- coding: ISO-8859-1 -*-

import datetime
import chardet
import csv
import json
import os
import re
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import time
import zipfile
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join
from os.path import splitext
from pandas import DataFrame as df
from cimaq_utils import flatten
from cimaq_utils import loadimages
from NEW_extract_events2 import NEW_extract_events2
from NEW_get_allsheetsdf import NEW_get_allsheetsdf
from chardet.universaldetector import UniversalDetector

def NEW_sniff_encodtype(maindir="~/extracted_eprime_AS_FWF"):
    ''' Detects encoding type used in given data table.
        Performs better with incremental runs.
        Source:
            https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
    '''
    if os.path.isdir(xpu(maindir)):
        allsheetsdf = NEW_get_allsheetsdf(join(maindir, 'participants.tsv'))
    else:
        allsheetsdf = allsheetsdf
    results = []
    for row in allsheetsdf.iterrows():
        detector = UniversalDetector()
        detector.reset()
        for line in open(row[1]['sheetpath'], 'rb'):
            detector.feed(line)
            if detector.done: break
        detector.close()
        results.append(pd.Series(detector.result, name=row[0]).squeeze())
    rezz = df(results).drop('language', axis=1)

    return rezz

def main():
    sniffed = NEW_sniff_encodtype()

if __name__ == '__main__':
    main()

In [ ]:
sniffed = NEW_sniff_encodtype()